In [2]:
# Snowpark for Python
from snowflake.snowpark.session import Session
#from snowflake.snowpark.functions import month,year,col,sum
#from snowflake.snowpark.version import VERSION
# Misc
import json
import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)

In [3]:
param = json.load(open('connection_movie.json'))

In [20]:
session = Session.builder.configs(param).create()

In [4]:
session

In [5]:
session.sql_simplifier_enabled = True

In [5]:
snowflake_environment = session.sql('select current_user(), current_version()').collect()

In [6]:
session.get_current_role()

'"ACCOUNTADMIN"'

In [7]:
session.get_current_database(), session.get_current_schema(), session.get_current_warehouse()

('"MOVIE_DB"', '"MOVIE_SCHEMA"', '"MOVIE_L"')

In [8]:
tables = session.sql('show tables').collect()

In [39]:
tables[0]['name']

'ITEMS'

In [9]:
for table in tables:
    print(table['name'].lower())

items
ratings
users


In [10]:
dict_of_vars = vars()

In [11]:
# dict_of_vars = vars()
# tables from DB to data frames
list_of_table_names = []
for table in tables:
    table_name = table['name'].lower()
    list_of_table_names.append(table_name)
    # converts strings items, users, retings into variables with the same name
    # saving a data frame object
    dict_of_vars[table_name] = session.table(table_name)

In [12]:
list_of_table_names

['items', 'ratings', 'users']

In [13]:
items.show(2)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ITEM_ID"  |"TITLE"           |"RELEASE_DATE"  |"IMDB_URL"                                          |"UNKNOWN_GENRE"  |"ACTION"  |"ADVENTURE"  |"ANIMATION"  |"CHILDREN"  |"COMEDY"  |"CRIME"  |"DOCUMENTARY"  |"DRAMA"  |"FANTASY"  |"FILM_NOIR"  |"HORROR"  |"MUSICAL"  |"MYSTERY"  |"ROMANCE"  |"SCI_FI"  |"THRILLER"  |"WAR"  |"WESTERN"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
users.show(2)

-------------------------------------------------------
|"USER_ID"  |"AGE"  |"GENDER"  |"OCCUPATION"  |"ZIP"  |
-------------------------------------------------------
|1          |24     |M         |technician    |85711  |
|2          |53     |F         |other         |94043  |
-------------------------------------------------------



In [15]:
ratings.show(2)

--------------------------------------------------
|"USER_ID"  |"ITEM_ID"  |"RATING"  |"TIMESTAMP"  |
--------------------------------------------------
|196        |242        |3.0       |881250949    |
|186        |302        |3.0       |891717742    |
--------------------------------------------------



In [16]:
items.queries

{'queries': ['SELECT  *  FROM (items)'], 'post_actions': []}

In [23]:
items.explain()

Failed to execute query [queryID: None] explain using text  SELECT  *  FROM (items)
390114 (08001): Authentication token has expired.  The user must authenticate again.


 SELECT  *  FROM (items) can't be explained
--------------------------------------------
